In [6]:
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
import json
import re
import time
import pandas as pd

In [4]:
uri = "http://localhost:7474"
pwd = "root123"
graph = Graph(uri, auth=("neo4j", pwd), name="dbc")

In [5]:
graph.run("MATCH (n) RETURN COUNT(n)")

COUNT(n)
2128


In [105]:
#f=open("/Users/thor/Git/ML/virk/xml/test/outrun2","r")
f=open("/Users/thor/Git/ML/virk/xrun/testout","r")

In [106]:
lines=f.readlines()

In [112]:
logfile=open("logtest.txt","w+")

In [113]:
print(len(lines))
print(lines[0])

300
00077201_109.xml.out:  <c:IdentificationNumberCvrOfAuditFirm contextRef="c1">18061635</c:IdentificationNumberCvrOfAuditFirm>



XVergoHoldingIVS FOOD-FACTORY ApS<br>
XIMTEXApS IMTEX ApS<br>


In [114]:
personDict={}
companyDict={}
listOfPatterns=[]
idPat=re.compile("(^[0-9_]+)")
cvrPat=re.compile("(^[0-9]+)")
idDir=re.compile("NameAndSurnameOfMemberOfExecutiveBoard.*>(.*)<\/.*")
idRev=re.compile("NameOfAuditFirm.*>[;]?(.*)<\/.*")
idComp=re.compile("NameOfReportingEntity.*>(.*)<\/.*")
idRevName=re.compile("NameAndSurnameOfAuditor.*>(.*)<\/.*")
idCVRAud=re.compile("IdentificationNumberCvrOfAuditFirm.*>(.*)<\/")
idLoss=re.compile("e:GrossProfitLoss.*>(.*)<\/")
idAddrS=re.compile("AddressOfReportingEntityStreetName.*>(.*)<\/")
idAddrN=re.compile("AddressOfReportingEntityStreetBuildingIdentifier.*>(.*)<\/")
idAddrC=re.compile("AddressOfReportingEntityDistrictName.*>(.*)<\/")
idCVR=re.compile("IdentificationNumberCvrOfReportingEntity.*>([0-9]+)<\/")
idBank=re.compile("NameOfFinancialInstitution.*>(.*)<\/")
idPerson=re.compile("NameOfSubmittingEnterprise.*>(.*)<\/")
grossLoss=re.compile("e:GrossProfitLoss.*>(.*)<\/")
idAddrSN=re.compile("AddressOfReportingEntityStreetAndNumber.*>(.*)<\/")



compDict={}
compDictIdxCounter=0
tmpcvr=0
previdx=0
idx=re.compile("([0-9]+)_([0-9]+)\.")
for line in lines:
    #print(line)
    
    # 00540001_202.xml.out: 
    # 00540001_384.xml.out:

    res=idx.findall(line)
    cvr=res[0][0]
    cnt=res[0][1]
    
    if cvr not in compDict.keys():
        #first line in next company
        compDict[cvr]={}
        compDictIdxCounter=cnt
        compDict[cvr]['GrossLoss']=0

    elif compDictIdxCounter != cnt:
        #logfile.write("DEBUG: new audit of same firm\n")
        pass
    #look for stuff
    else:
        #compDictIdxCounter+=1           
        idCvrL=idPat.findall(line)
        idCvrL=cvrPat.findall(line)
        idDirL=idDir.findall(line)
        idRevL=idRev.findall(line)
        idCompL=idComp.findall(line)
        idRevNameL=idRevName.findall(line)
        idAddrSL=idAddrS.findall(line)
        idAddrNL=idAddrN.findall(line)
        idAddrCL=idAddrC.findall(line)
        idCVRL=idCVR.findall(line)
        idBankL=idBank.findall(line)
        idPersonL=idPerson.findall(line)
        grossLossL=grossLoss.findall(line)

        if len(grossLossL)>0:
            compDict[cvr]['GrossLoss']=grossLossL[0]
      
        if len(idPersonL)>0:
            compDict[cvr]['NameOfCompPerson']=idPersonL[0]
            compDict[cvr]['idNameOfCompPerson']="{0}".format(re.sub("[ -\.\/;&,'´]","",idPersonL[0]))

        if len(idBankL)>0:
            compDict[cvr]['NameOfBank']=idBankL[0]
            compDict[cvr]['idNameOfBank']="{0}".format(re.sub("[ -\.\/;&,'´]","",idBankL[0]))

        if len(idDirL)>0:
            compDict[cvr]['NameOfCEO']=idDirL[0]
            compDict[cvr]['idNameOfCEO']="{0}".format(re.sub("[ -\.\/;&,'´]","",idDirL[0]))
        if len(idRevL)>0:
            compDict[cvr]['NameOfAuditFirm']=re.sub("[']","",idRevL[0])
            tmpStr=re.sub("[ -\.\/;&,'´]","",idRevL[0])
            if tmpStr[0].isdigit():
                nStr="X"+tmpStr
            else:
                nStr=tmpStr       
            compDict[cvr]['idNameOfAuditFirm']=nStr
        if len(idCompL)>0:
            compDict[cvr]['NameOfReportingEntity']=re.sub("^[0-9]|[']","",idCompL[0])
            tmpStr=re.sub("[ -\.\/;&,´]","",idCompL[0])
            tmpStrN=tmpStr
            if tmpStr[0].isnumeric():
                tmpStrN="X"+tmpStr     
            compDict[cvr]['idNameOfReportingEntity']=re.sub("[ -\.\/;&,'´]","",tmpStrN)
        if len(idRevNameL)>0:
            compDict[cvr]['NameAndSurnameOfAuditor']=idRevNameL[0]
            compDict[cvr]['idNameAndSurnameOfAuditor']=re.sub("[ -\.\/;&,'´]","",idRevNameL[0])
        if len(idAddrSL)>0:
            if ("location" not in compDict[cvr].keys()):
                compDict[cvr]['location']={}       
            compDict[cvr]['location']['Street']=idAddrSL[0]
            if ('idLocation' in compDict[cvr]['location'].keys() ):
                compDict[cvr]['location']['idLocation']+=re.sub("[ -\.\/;&,'´]","",idAddrSL[0])
            else:
                compDict[cvr]['location']['idLocation']=re.sub("[ -\.\/;&,'´]","",idAddrSL[0])
        if len(idAddrNL)>0:
            if ("location" not in compDict[cvr].keys()):
                compDict[cvr]['location']={}
            compDict[cvr]['location']['Number']=idAddrNL[0]
            if ('idLocation' in compDict[cvr]['location'].keys() ):
                compDict[cvr]['location']['idLocation']+=re.sub("[ -\.\/;&,'´]","",idAddrNL[0])
            else:
                compDict[cvr]['location']['idLocation']=re.sub("[ -\.\/;&,'´]","",idAddrNL[0])

        if len(idAddrCL)>0:
            try:
                if ("location" not in compDict[cvr].keys()):
                    compDict[cvr]['location']={}
                compDict[cvr]['location']['City']=idAddrCL[0]

                if ('idLocation' in compDict[cvr]['location'].keys() ):
                    tmpStrLoc=re.sub("[ -\.\/;&,'´]","",idAddrCL[0])
                    if (tmpStrLoc[0].isdigit()):
                        tmpStrLoc="Z"+tmpStrLoc
                    compDict[cvr]['location']['idLocation']+=re.sub("[ -\.\/;&,'´]","",idAddrCL[0])
                else:
                    compDict[cvr]['location']['idLocation']=re.sub("[ -\.\/;&,'´]","",idAddrCL[0])
            except:
                print("cont")
        if len(idCVRL)>0:
            compDict[cvr]['cvr']=idCVRL[0]

    #idDirL=idDir.findall(line)

    
    
    

In [83]:
compDict['99878001'].values()

KeyError: '99878001'

In [34]:
import ipdb

In [285]:
#create cypher
#if ceo

if ('idNameOfCEO' in compDict['01339483'].keys()):
    tmpStr="MERGE({0}:Person {{name:'{1}', role:'ceo'}}) MERGE ({2}:Company {{name:'{3}'}}) MERGE ({0})-[:CEO_IN]->({2})"
    q=tmpStr.format(
        compDict['01339483']['idNameOfCEO'],
        compDict['01339483']['NameOfCEO'],
        compDict['01339483']['idNameOfReportingEntity'],
        compDict['01339483']['NameOfReportingEntity']
    )
    

In [286]:
print(q)
#graph.run(q)

MERGE(HenrikLund-Nielsen:Person {name:'Henrik Lund-Nielsen', role:'ceo'}) MERGE (2084NOApS:Company {name:'2084.NO ApS'}) MERGE (HenrikLund-Nielsen)-[:CEO_IN]->(2084NOApS) MERGE(Gentofte23Soløsevej:Location {city:'Gentofte',street:'Soløsevej',number:'23'})  MERGE (2084NOApS)-[:LOCATED]->(Gentofte23Soløsevej)


In [230]:
if ("location" in compDict['01069173'].keys()):
    tmpStr=" MERGE({0}:Location {{city:'{1}',street:'{3}',number:'{2}'}})  MERGE ({4})-[:LOCATED]->({0})"
    q+=tmpStr.format(
        compDict['01069173']['location']['idLocation'],
        compDict['01069173']['location']['City'],
        compDict['01069173']['location']['Number'],
        compDict['01069173']['location']['Street'],
        compDict['01069173']['idNameOfReportingEntity']
    )
    

In [231]:
if ("idNameOfAuditFirm" in compDict['01069173'].keys()):
    tmpStr=" MERGE({0}:Person {{name:'{1}', role:'auditer'}}) MERGE ({2}:Company {{name:'{3}'}})  MERGE ({0})-[:EMP_IN]->({2}) MERGE ({4})-[:AUDITED_BY]->({0}) "
    q+=tmpStr.format(
        compDict['01069173']['idNameAndSurnameOfAuditor'],
        compDict['01069173']['NameAndSurnameOfAuditor'],
        compDict['01069173']['idNameOfAuditFirm'],
        compDict['01069173']['NameOfAuditFirm'],
        compDict['01069173']['idNameOfReportingEntity']
    )
    

In [115]:
for key in compDict.keys():
    if ('idNameOfReportingEntity') not in compDict[key].keys():
        compDict[key]['idNameOfReportingEntity']="NaN"
        logfile.write("ERROR on {}\n".format(compDict[key]))
    
    #ipdb.set_trace()
    
    if ('idNameOfCEO' in compDict[key].keys()):
        try:
            tmpStr="MERGE({0}:Person {{name:'{1}', role:'ceo'}}) MERGE ({2}:Company {{name:'{3}',grossloss:'{4}'}}) MERGE ({0})-[:CEO_IN]->({2})"
            q=tmpStr.format(
                compDict[key]['idNameOfCEO'],
                compDict[key]['NameOfCEO'],
                compDict[key]['idNameOfReportingEntity'],
                compDict[key]['NameOfReportingEntity'],
                compDict[key]['GrossLoss']
            )
        except Exception as e:
            logfile.write("ERROR:{}\n".format(e))
            logfile.write("ERROR on {}\n".format(compDict[key]))
            tmpStr="MERGE({0}:Person {{name:'{1}', role:'ceo'}})"
            q=tmpStr.format(
                compDict[key]['idNameOfCEO'],
                compDict[key]['NameOfCEO']
            )
            compDict[key]['idNameOfReportingEntity']="NaN"

    if ("location" in compDict[key].keys()):
        try:
            tmpStr=" MERGE({0}:Location {{city:'{1}',street:'{3}',number:'{2}'}})  MERGE ({4})-[:LOCATED]->({0})"
            q+=tmpStr.format(
                compDict[key]['location']['idLocation'],
                compDict[key]['location']['City'],
                compDict[key]['location']['Number'],
                compDict[key]['location']['Street'],
                compDict[key]['idNameOfReportingEntity']
            )
        except Exception as e:
            logfile.write("ERROR:{}\n".format(e))
            logfile.write("ERROR on {}\n".format(compDict[key]))
            tmpStr="MERGE({0}:Person {{name:'{1}', role:'ceo'}})"
            tmpStr=" MERGE({0}:Location)  MERGE ({1})-[:LOCATED]->({0})"
            q+=tmpStr.format(
                compDict[key]['location']['idLocation'],
                compDict[key]['idNameOfReportingEntity']
            )

    if ("idNameOfAuditFirm" in compDict[key].keys()):
        try:
            tmpStr=" MERGE({0}:Person {{name:'{1}', role:'auditer'}}) MERGE ({2}:Company {{name:'{3}'}})  MERGE ({0})-[:EMP_IN]->({2}) MERGE ({4})-[:AUDITED_BY]->({0}) "
            q+=tmpStr.format(
                compDict[key]['idNameAndSurnameOfAuditor'],
                compDict[key]['NameAndSurnameOfAuditor'],
                compDict[key]['idNameOfAuditFirm'],
                compDict[key]['NameOfAuditFirm'],
                compDict[key]['idNameOfReportingEntity']
            )
        except:
            tmpStr=" MERGE ({0}:Company {{name:'{1}'}})  MERGE ({2})-[:AUDITED_BY]->({0}) "

            q+=tmpStr.format(
            compDict[key]['idNameOfAuditFirm'],
            compDict[key]['NameOfAuditFirm'],
            compDict[key]['idNameOfReportingEntity']
            )
        

    try:
        graph.run(q)
        logfile.write("DEBUG: {}\n".format(q))

    except Exception as e:
        logfile.write("ERROR: {}\n".format(e))
        logfile.write("ERROR Q: {} \n".format(q))
     
    logfile.flush()
    #print(q)

In [111]:
logfile.close()

In [357]:
!pwd

/Users/thor/Git/bite_sized_data_science/notebooks


In [235]:
print(q)
#print(data[0])
#q="CREATE (ElseH:creator {name:'Else Holgersen'}) CREATE (Fljteogmalebogen:sheetmusic {title:'Fløjte og malebogen', subjectterm:['78.46301', 'Vokalmusik for børn']}),(ElseH)-[:CREATED]->(Fljteogmalebogen)"

MERGE(TorstenWoldbyeMandrup:Person {name:'Torsten Woldbye Mandrup', role:'ceo'}) MERGE (KNMachinesIVS:Company {name:'KN Machines IVS'}) MERGE (TorstenWoldbyeMandrup)-[:CEO_IN]->(KNMachinesIVS) MERGE(Horsens6Toftumvej:Location {city:'Horsens',street:'Toftumvej',number:'6'})  MERGE (KNMachinesIVS)-[:LOCATED]->(Horsens6Toftumvej) MERGE(MogensNygård:Person {name:'Mogens Nygård', role:'auditer'}) MERGE (MNRevisionogRådgivning:Company {name:'MN Revision og Rådgivning'})  MERGE (MogensNygård)-[:EMP_IN]->(MNRevisionogRådgivning) MERGE (KNMachinesIVS)-[:AUDITED_BY]->(MogensNygård) 


In [236]:
graph.run(q)

(No data)

In [ ]:
for dat in data:
    try:
        print((dat['type']))
    except:
        print("Error on {}".format(dat))
    

In [55]:
f.close()

In [49]:
print(data)

[]
